# Settings

In [ ]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep
import sys
sys.path.append('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..') # For Windows import
os.chdir('..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..') # For Linux import
print(sep_local)
print(os.getcwd())


In [ ]:
import tensorflow as tf
print(tf.__version__)

# Dataset loading

In [ ]:
dataset_name='atari_pacman'

In [ ]:
IMG_DIR = '/home/azeghost/datasets/.mspacman/atari_v1/screens/mspacman' #Linux
#IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 25
VAL_FORMAT = 'png'

In [ ]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [ ]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

In [ ]:
# scale=1
# IMG_SIZE=(160//scale, 210//scale, 3)
# BATCH_SIZE = 5
# EPIS_LEN = 10
# EPIS_SHIFT = 5


# inputs_shape= IMG_SIZE
# latent_dim = 10
# intermediate_dim = 10

scale=1
IMG_SIZE=(160//scale, 210//scale, 3)

BATCH_SIZE = 10
EPIS_LEN = 10
EPIS_SHIFT = 5

inputs_shape= IMG_SIZE
latent_dim = 30
intermediate_dim = 30

In [ ]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode='episode_flat', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE* EPIS_LEN, ) + IMG_SIZE)
                  )
)

In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [ ]:
_instance_scale=1.0

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

# Model's Layers definition

In [ ]:
# _uf=100
# c=16
_uf=30
# c=(160//4, 210//6, intermediate_dim//2)
c=(160//4, 210//6, intermediate_dim//2)
menc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf//5, kernel_size=3, strides=(2, 3), activation='relu'), # _uf*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf//5, kernel_size=3, strides=(2, 3), activation='relu'), #_uf*9
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=np.product(c), activation=tf.nn.relu), # units=_uf*c*c
    tf.keras.layers.Reshape(target_shape=c), #target_shape=(c , c, _uf)
    tf.keras.layers.Conv2DTranspose(filters=_uf//5, kernel_size=3, strides=(2, 3), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=_uf, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [ ]:
model_name = dataset_name+'Conv_VAE'
#windows
#recoding_dir='..' + sep_local + '..' + sep_local +'..' + sep_local + '..' + sep_local + '..'+sep_local+'recording'+sep_local + model_name

#linux 
recoding_dir=os.getcwd()+ sep_local  +'recording'+sep_local + model_name

In [ ]:
from training.traditional.transformative.VAE import VAE as AE

In [ ]:
variables_params = \
[
    {
        'name': 'encoder_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays
    }

    ,
    
        {
        'name': 'encoder_logvar', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from os.path import abspath
from utils.data_and_files.file_utils import create_if_not_exist
_restore = os.path.join(recoding_dir, 'var_save_dir')
create_if_not_exist(_restore)
absolute = abspath(_restore)
print("Restore_dir",absolute)
absolute = abspath(recoding_dir)
print("Recording_dir",absolute)
print("Current working dir",os.getcwd())

In [ ]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE* EPIS_LEN,
    variables_params=variables_params, 
    filepath=None #to restore trained model, set filepath=_restore
    )

In [ ]:
ae.compile()

# Callbacks

In [ ]:
# added for linux warning suppression
import logging
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
progbar = NotebookPrograssBar(leave_outer=False)

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [ ]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [ ]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, model_name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
absolute = abspath(csv_dir)
print("Csv_dir",absolute)

In [ ]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)
absolute = abspath(image_gen_dir)
print("Image_gen_dir",absolute)

In [ ]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)

# prediction preview

In [ ]:
for batch in test_ds:
    break

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xt0 = batch[0][:10]
xt1 = batch[1][:10]

In [ ]:
xt0 = tf.concat([x for x in xt0], axis=1)
xt1 = tf.concat([x for x in xt1], axis=1)

In [ ]:
xt1_pred = ae.predict(batch[0][:10])
xt1_pred = tf.concat([x for x in xt1_pred], axis=1)

In [ ]:
plt.figure(figsize=(20, 80), dpi=300)
plt.axis('off')
print('input')
plt.imshow(xt0)
#plt.save()
plt.show()


plt.figure(figsize=(20, 80), dpi=300)
plt.axis('off')
print('output')
plt.imshow(xt1)
#plt.save()
plt.show()

plt.figure(figsize=(20, 80), dpi=300)
plt.axis('off')
print('prediction')
plt.imshow(xt1_pred)
#plt.save()
plt.show()
